In [1]:
'''
    spam.py builds a spam filter based on Paul Graham's A Plan for Spam.  CS 344 Calvin College

 @author:  austin gibson
 @version: March 6, 2019

'''


spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "I", "like", "green", "eggs", "and", "ham"], ["I", "do"]]

test_mail1 = ["I", "am", "spam", "but", "if", "I", "like", "green", "eggs", "and", "ham", "am", "I", "still", "spam"]
test_mail2 = ["I", "am", "spam", "but", "if", "I", "like", "green", "eggs", "and", "ham", "am", "I", "still", "spam",
                "no", "unless", "theres", "extra", "spam", "I", "am"]

#create hash tables
good = {}
bad = {}

#loop through corpuses and create hash tables with number of occurences
for i in range(len(spam_corpus)):
    for j in range(len(spam_corpus[i])):
        if spam_corpus[i][j] in bad.keys():
            bad[spam_corpus[i][j]] = bad[spam_corpus[i][j]] + 1
        else:
            bad[spam_corpus[i][j]] = 1

for i in range(len(ham_corpus)):
    for j in range(len(ham_corpus[i])):
        if ham_corpus[i][j] in good.keys():
            good[ham_corpus[i][j]] = good[ham_corpus[i][j]] + 1
        else:
            good[ham_corpus[i][j]] = 1

#ngood and nbad are the number of nonspam & spam messages
ngood = 0
nbad = 0

for key in good:
    ngood += good[key]

for key in bad:
    nbad += bad[key]

#calculate the probability email containing it is a spam
def token_prob(word):
    try:
        g = 2 * good[word]
    except:
        g = 0
    try:
        b = bad[word]
    except:
        b = 0
    if g + b > 1:
        return max(0.01, min(.99, min(1.0, b/nbad) / (min(1.0, g/ngood) + min(1.0, b/nbad))))
    else:
        return 0

#create third hash to map token to probability
probhash = {}
for word in good:
    probhash[word] = token_prob(word)
for word in bad:
    probhash[word] = token_prob(word)


#getting new mail
def newMail(mailList):
    probList = []
    #get probability for tokens in new mail.  .4 if no probabilty
    if len(mailList) <= 15:
        for word in mailList:
            prob = token_prob(word)
            if prob != 0:
                probList.append(prob)
            else:
                probList.append(.4)
    # if mail tokens > 15.  Get 15 most interesting which is distance from .5
    else:
        unsortedList=[]
        for word in mailList:
            prob = token_prob(word)
            if prob != 0:
                unsortedList.append([prob, abs(prob - .5)])
            else:
                unsortedList.append([.4, abs(.4 - .5)])
        unsortedList.sort(key=lambda x: x[1], reverse=True)
        for i in range(15):
            probList.append(unsortedList[i][0])
    return probList

#determine if newMail is spam
def probability(listofProbs):
    prod = 1
    compliments = 1
    for prob in listofProbs:
        prod *= prob
        compliments *= 1 - prob
    return prod / (prod + compliments)

#print out hash tables, ngood, nbad
print("hash of good tokens: ")
print(good)
print("\n hash of bad tokens: ")
print(bad)
print("\n probability hash table: ")
print(probhash)
print("\n ngood: " + str(ngood))
print(" nbad: " + str(nbad))

#test new mail
newmail = newMail(test_mail1)
print("\n new mail: ")
print(test_mail1)
print("final probability mail is spam: " + str(probability(newmail)))

#test mail longer than len 15
newmail = newMail(test_mail2)
print("\n new mail: ")
print(test_mail2)
print("final probability mail is spam: " + str(probability(newmail)))

hash of good tokens: 
{'ham': 1, 'do': 2, 'eggs': 1, 'I': 2, 'like': 1, 'green': 1, 'and': 1}

 hash of bad tokens: 
{'that': 1, 'not': 1, 'do': 1, 'am': 2, 'I': 3, 'spam': 2, 'like': 1, 'spamiam': 1}

 probability hash table: 
{'that': 0, 'spamiam': 0, 'eggs': 0.01, 'like': 0.27272727272727276, 'not': 0, 'ham': 0.01, 'do': 0.15789473684210525, 'I': 0.36, 'spam': 0.99, 'green': 0.01, 'and': 0.01, 'am': 0.99}

 ngood: 9
 nbad: 12

 new mail: 
['I', 'am', 'spam', 'but', 'if', 'I', 'like', 'green', 'eggs', 'and', 'ham', 'am', 'I', 'still', 'spam']
final probability mail is spam: 0.019391908067991324

 new mail: 
['I', 'am', 'spam', 'but', 'if', 'I', 'like', 'green', 'eggs', 'and', 'ham', 'am', 'I', 'still', 'spam', 'no', 'unless', 'theres', 'extra', 'spam', 'I', 'am']
final probability mail is spam: 0.997289624040641


In [3]:
'''
This module implements the Bayesian network shown in figure 14.12.
For homework2, cs344 at Calvin College

@author: austin gibson
@version March 6, 2019
'''

from probability import BayesNet, enumeration_ask
#elimination_ask, gibbs_ask

# Utility variables
T, F = True, False

cloudy = BayesNet([
    ('Cloudy', '', 0.5),
    ('Sprinkler', 'Cloudy', {T: 0.1, F: 0.5}),
    ('Rain', 'Cloudy', {T: 0.8, F: 0.2}),
    ('WetGrass', 'Sprinkler Rain', {(T, T): 0.99, (T, F): 0.9, (F, T): 0.9, (F, F): 0.00}),
    ])


"""
2b. Independent values in full joint
     size = s^ N
     n = 4 variables
     2 ^ 4 = 16

2c. Independent values in bayesnet for domain
    Referencing the BayesNet Implement earlier, there are 9 pairs.
    
"""

# Compute P(Cloudy)
print("\n P(Cloudy")
print(enumeration_ask('Cloudy', dict(), cloudy).show_approx())
"""
Calculations:
    P(Cloudy) = <.5 , .5> taken from figure 14.12
"""
# Compute P(Sprinkler | cloudy)
print("\n P(Sprinkler | cloudy")
print(enumeration_ask('Sprinkler', dict(Cloudy=T), cloudy).show_approx())
"""
Calculations:
    P(Cloudy) = <.1, .9> again taken from figure 14.12
"""

# Compute P(Cloudy | sprinkler ^ not raining)
print("\n P(Cloudy | sprinkler ^ not raining")
print(enumeration_ask('Cloudy', dict(Sprinkler=T, Rain=F), cloudy).show_approx())
"""
Calculations:
    P(Cloudy | sprinkler ^ not raining)
        = alpha * < P(C)P(s|C)P(-r|C), P(-C)P(s|-C)P(-r|-C)
        = alpha * <.5 *.1*.2, .5*.5*.8>
        = alpha * <.01, .2>
        = .01 / (.01 + .2) , .2/ .21
        = < .0476, .952>
"""

# Compute P(WetGrass | cloudy & sprinkler & rain)
print("\n P(WetGrass | cloudy & sprinkler & rain")
print(enumeration_ask('WetGrass', dict(Cloudy=T, Sprinkler=T, Rain=T), cloudy).show_approx())
"""
Calculations:
    P(WetGrass | cloudy & sprinkler & r)
        = alpha * p(c) < P(s|c)P(r|c)*P(wg|sr), P(s|C)P(r|C)P(-wg|sr)>
        = alpha * .5 * <.1*.8*.99, .1*.8*.01>
        = alpha * .5 * < .0792 , .008>
        = <.0396 , .004>
        = .0396 / (.0396 + .0004) , .004 / .04
        = <.99 , .01>
"""
# Compute P(Cloudy | grass not wet)
print("\n P(Cloudy | grass not wet")
print(enumeration_ask('Cloudy', dict(WetGrass=F), cloudy).show_approx())
"""
Calculations:
    P(Cloudy | grass not wet)
        = alpha *P(C) <*P(sr|C)*P(-gw|sr) + P(-sr|C)P(-gw|-sr)+P(s-r|C)P(-gw|s-r)+P(-s-r|C)P(-gw|-s-r), same but not C>
        = alpha * .5 * <.08*.01 + .72*.1+ .02*.1+.18*1, .1*.01+.4*.1+.1*.1+.4*1>
        = alpha * .5 * <.2548 , .451>
        = <.1274 , .2255>
        = .1274 / (.1274+.2255) , .2255 / .3529
        = <.361 , .639>
"""


 P(Cloudy
False: 0.5, True: 0.5

 P(Sprinkler | cloudy
False: 0.9, True: 0.1

 P(Cloudy | sprinkler ^ not raining
False: 0.952, True: 0.0476

 P(WetGrass | cloudy & sprinkler & rain
False: 0.01, True: 0.99

 P(Cloudy | grass not wet
False: 0.639, True: 0.361


'\nCalculations:\n    P(Cloudy | grass not wet)\n        = alpha *P(C) <*P(sr|C)*P(-gw|sr) + P(-sr|C)P(-gw|-sr)+P(s-r|C)P(-gw|s-r)+P(-s-r|C)P(-gw|-s-r), same but not C>\n        = alpha * .5 * <.08*.01 + .72*.1+ .02*.1+.18*1, .1*.01+.4*.1+.1*.1+.4*1>\n        = alpha * .5 * <.2548 , .451>\n        = <.1274 , .2255>\n        = .1274 / (.1274+.2255) , .2255 / .3529\n        = <.361 , .639>\n'